In [17]:
import requests
import csv

# Globals
BIOLINK_API = "https://api-dev.monarchinitiative.org/api"
NUM_TERMS = 4

# Input/output files
file_path = "/home/kshefchek/Downloads/OmsLogFindingsSynonyms.csv"
output_file = "/home/kshefchek/Oms-HPO-terms.tsv"

outfile = open(output_file, 'w')

columns = ['finding label', 'synonym label', 'matched label', 'matched id', 'matched text']

outfile.write("#{}\n".format("\t".join(columns)))

biolink_search = BIOLINK_API + "/search/entity/autocomplete/"

parameters = {
    'rows': NUM_TERMS,
    'prefix': 'HP'
}

with open(file_path, 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=',', quotechar='\"')
    next(reader, None)  # skip the headers
    for row in reader:
        if row[0].startswith('#'): continue
        (term, synonym) = row
              
        outfile.write("{0}\t{1}\t".format(term, synonym))
        
        url = biolink_search + term
        biolink_request = requests.get(url, params=parameters)
        biolink_response = biolink_request.json()
        
        for match in biolink_response['docs']:
            label = match['label'][0]
            match_id = match['id']
            matched_text = match['match']
            outfile.write("{}\t{}\t{}\t".format(label, match_id, matched_text))
        
        
        outfile.write("\n")

outfile.close()
